In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [1]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip3 install pandas duckdb scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!python --version

Python 3.7.15


In [102]:
import os
import time
import numpy as np
import pandas as pd
import duckdb
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from IPython.display import clear_output

In [4]:
from google.colab import drive
drive.mount('/content/drive')
wd = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'GT', 'bd4h_proj')
os.chdir(wd)
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/GT/bd4h_proj'

In [5]:
con = duckdb.connect()

In [34]:
sparse_array = np.load('i2b2_sparse_feature.npy', allow_pickle=True)
# sparse_dict.items()
# sparse_array = sparse_dict['arr_0']
# for k, v in sparse_dict.items():
#   sparse_array = sparse_dict[k]
#   break
type(sparse_array)

numpy.ndarray

In [35]:
sparse_array.shape

(1088, 15453)

In [38]:
feature_df = pd.DataFrame(sparse_array).reset_index().rename(columns={'index': 'SUBJECT_ID'})
feature_df.head()

,SUBJECT_ID,0,1,2,3,4,5,6,7,8,...,15443,15444,15445,15446,15447,15448,15449,15450,15451,15452
0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# feature_df = pd.read_csv('patient_feature.csv')
# feature_df.head(2)

In [40]:
target_df = pd.read_csv('patient_target.csv')
target_df.head()

,SUBJECT_ID,1,2,3,4,5,6,7,8,9,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
feature_df = con.execute("""
select *
from feature_df f 
where exists (select 1 from target_df t 
  where f.subject_id = t.subject_id)
order by subject_id
""").df().reset_index()
feature_df.head()

,index,SUBJECT_ID,0,1,2,3,4,5,6,7,...,15443,15444,15445,15446,15447,15448,15449,15450,15451,15452
0,0,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
target_df = con.execute("""
select *
from target_df t
where exists (select 1 from feature_df f
  where f.subject_id = t.subject_id)
order by subject_id
""").df().reset_index()
target_df.head(2)

,index,SUBJECT_ID,1,2,3,4,5,6,7,8,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
def set_device():
  print(f"cuda: {torch.cuda.is_available()}")
  device = torch.device("cuda")
  return device
device = set_device()
torch.manual_seed(1)
if device.type == "cuda":
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

cuda: True


In [49]:
RESERVED_COLS = 2
SEED = 42

In [90]:
num_obs = feature_df.count()[0]

In [122]:
feature = torch.tensor(feature_df.iloc[:, RESERVED_COLS:].values, dtype=torch.float)
target = torch.tensor(target_df.iloc[:, RESERVED_COLS:].values, dtype=torch.float)
dataset = TensorDataset(feature, target)
indices = list(range(num_obs))
# train_size = int(0.8 * num_obs)
train_size = int(1 * num_obs)
valid_size = num_obs - train_size
split = [train_size, valid_size]
gen = torch.Generator().manual_seed(SEED)
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, split, 
                                                  generator=gen)

In [123]:
BATCH_SIZE = 1021
feature_count = len(feature_df.columns) - RESERVED_COLS
target_count = len(target_df.columns) - RESERVED_COLS
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [124]:
class PatientReprMlp(nn.Module):
	def __init__(self, feature_count=feature_count, 
                        target_count=target_count):
		super(PatientReprMlp, self).__init__()
		self.input_dim = feature_count
		self.hidden_size = 1000
		self.num_labels = target_count
		self.p = 0.2
		self.dropout = nn.Dropout(self.p)
		self.fc1 = nn.Linear(self.input_dim, self.hidden_size)
		self.act = nn.ReLU()
		self.fc2 = nn.Linear(self.hidden_size, self.num_labels)

		# self.linear_layer = nn.Sequential(
    #         nn.Dropout(self.p),
		# 	nn.Linear(self.input_dim, self.hidden_size),
    #         nn.ReLU(),
		# 	nn.Linear(self.hidden_size, self.num_labels)
    #     )

	def forward(self, x):
		x = self.dropout(x)
		x = self.act(self.fc1(x))
		x = self.fc2(x)
		# for layer in self.linear_layer:
		# 	x = layer(x)
		return x

In [125]:
NUM_EPOCHS = 100
model = PatientReprMlp()
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
device = set_device()
model.to(device)
criterion.to(device)
save_file = 'model.pth'

cuda: True


In [126]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.fc1.register_forward_hook(get_activation('fc1'))    

In [127]:
class AverageMeter(object):
	"""Computes and stores the average and current value"""

	def __init__(self):
		self.reset()

	def reset(self):
		self.val = 0
		self.avg = 0
		self.sum = 0
		self.count = 0

	def update(self, val, n=1):
		self.val = val
		self.sum += val * n
		self.count += n
		self.avg = self.sum / self.count


def compute_batch_accuracy(output, target):
	"""Computes the accuracy for a batch"""
	# print(f'output {output.size()}')
	# print(f'target {target.size()}')
	with torch.no_grad():
		batch_size = target.size(0)
	
		correct = output.eq(target).sum()	
		# _, pred = output.max(1)
		# print(pred.size())
		# correct = pred.eq(target).sum()
		# print(correct)

		return correct * 100.0 / batch_size


def train(model, device, data_loader, criterion, optimizer, epoch, print_freq=10):
	batch_time = AverageMeter()
	data_time = AverageMeter()
	losses = AverageMeter()
	accuracy = AverageMeter()

	model.train()

	end = time.time()
	for i, (input, target) in enumerate(data_loader):
		# measure data loading time
		data_time.update(time.time() - end)

		if isinstance(input, tuple):
			input = tuple([e.to(device) if type(e) == torch.Tensor else e for e in input])
		else:
			input = input.to(device)
		target = target.to(device)

		optimizer.zero_grad()
		output = model(input)
		loss = criterion(output, target)
		assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

		loss.backward()
		optimizer.step()

		# measure elapsed time
		batch_time.update(time.time() - end)
		end = time.time()

		losses.update(loss.item(), target.size(0))
		accuracy.update(compute_batch_accuracy(output, target).item(), target.size(0))

		if i % print_freq == 0:
			clear_output(wait=True)
			print('Epoch: [{0}][{1}/{2}]\t'
				  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
				  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
				  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
				  'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
				epoch, i, len(data_loader), batch_time=batch_time,
				data_time=data_time, loss=losses, acc=accuracy))

	return losses.avg, accuracy.avg


def evaluate(model, device, data_loader, criterion, print_freq=10):
	batch_time = AverageMeter()
	losses = AverageMeter()
	accuracy = AverageMeter()

	results = []

	model.eval()

	with torch.no_grad():
		end = time.time()
		for i, (input, target) in enumerate(data_loader):

			if isinstance(input, tuple):
				input = tuple([e.to(device) if type(e) == torch.Tensor else e for e in input])
			else:
				input = input.to(device)
			target = target.to(device)

			output = model(input)
			loss = criterion(output, target)

			# measure elapsed time
			batch_time.update(time.time() - end)
			end = time.time()

			losses.update(loss.item(), target.size(0))
			accuracy.update(compute_batch_accuracy(output, target).item(), target.size(0))

			y_true = target.detach().to('cpu').numpy().tolist()
			y_pred = output.detach().to('cpu').max(1)[1].numpy().tolist()
			results.extend(list(zip(y_true, y_pred)))

			if i % print_freq == 0:
				clear_output(wait=True)
				print('Test: [{0}/{1}]\t'
					  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
					  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
					  'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
					i, len(data_loader), batch_time=batch_time, loss=losses, acc=accuracy))

	return losses.avg, accuracy.avg, results

In [105]:
feature_df.values.shape, target_df.values.shape

((1021, 15455), (1021, 2805))

In [128]:
best_val_acc = 0.0
train_losses, train_accuracies = [], []
valid_losses, valid_accuracies = [], []

for epoch in range(NUM_EPOCHS):
	train_loss, train_accuracy = train(model, device, train_loader, criterion, optimizer, epoch)
	valid_loss, valid_accuracy, valid_results = evaluate(model, device, valid_loader, criterion)

	train_losses.append(train_loss)
	valid_losses.append(valid_loss)

	train_accuracies.append(train_accuracy)
	valid_accuracies.append(valid_accuracy)

	is_best = valid_accuracy > best_val_acc  # let's keep the model that has the best accuracy, but you can also use another metric.
	if is_best:
		best_val_acc = valid_accuracy
		# torch.save(model, save_file)


Epoch: [99][0/1]	Time 0.163 (0.163)	Data 0.130 (0.130)	Loss 0.6865 (0.6865)	Accuracy 0.000 (0.000)


After training on the input, run one epoch on the eval set to get the dense feature vectors.

In [111]:
print(activation['fc1'][0][0:10])

tensor([-0.0418,  0.0073, -0.0689, -0.0155,  0.0365,  0.1054, -0.0269, -0.0506,
         0.0166,  0.1844], device='cuda:0')


In [129]:
dense_feature_arr = activation['fc1'].cpu().numpy()
dense_feature_arr.shape

(1021, 1000)

In [130]:
feature_cols = []
for i in range(dense_feature_arr.shape[1]):
  feature_cols.append(f'F{i}')

feature_cols[:5]

['F0', 'F1', 'F2', 'F3', 'F4']

In [131]:
dense_feature_df = pd.DataFrame(dense_feature_arr, columns = feature_cols).reset_index()
dense_feature_df.head()

,index,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,F990,F991,F992,F993,F994,F995,F996,F997,F998,F999
0,0,0.052208,0.105053,-0.100350,-0.042010,0.038894,0.086565,-0.013481,0.092487,-0.137238,...,-0.117032,0.095805,-0.065410,0.052102,0.011810,0.043094,0.161437,-0.060491,0.045529,-0.112069
1,1,-0.026232,0.133410,-0.062311,-0.087948,0.065563,0.046006,0.000258,0.041759,-0.084532,...,-0.027079,0.108531,-0.076217,-0.012878,-0.154045,0.055077,-0.005913,-0.047781,0.008047,0.002012
2,2,-0.070908,-0.068187,-0.048338,-0.148972,0.046354,0.132966,0.026155,-0.030616,-0.017034,...,0.066733,0.121087,-0.008704,0.005541,-0.010806,0.072195,0.054540,-0.076624,-0.072656,0.062275
3,3,0.015193,0.108114,-0.059156,-0.062374,0.093193,0.095682,0.023337,-0.033058,-0.080769,...,0.148111,0.098194,0.032282,0.187567,0.015080,0.002625,0.079945,-0.008672,0.012438,-0.078997
4,4,0.120460,0.116227,0.073314,-0.092273,-0.034998,0.050711,-0.036398,0.054734,-0.137434,...,0.046523,0.110233,-0.028940,0.135374,0.017890,0.008272,0.142290,0.130462,0.147706,0.051248


In [132]:
dense_patient_feature_df = con.execute("""
select f.SUBJECT_ID, d.*
from dense_feature_df d
join feature_df f
on d.index = f.index
""").df()
dense_patient_feature_df = dense_patient_feature_df.drop(columns=['index'])
dense_patient_feature_df.head()

,SUBJECT_ID,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,F990,F991,F992,F993,F994,F995,F996,F997,F998,F999
0,2,0.052208,0.105053,-0.100350,-0.042010,0.038894,0.086565,-0.013481,0.092487,-0.137238,...,-0.117032,0.095805,-0.065410,0.052102,0.011810,0.043094,0.161437,-0.060491,0.045529,-0.112069
1,3,-0.026232,0.133410,-0.062311,-0.087948,0.065563,0.046006,0.000258,0.041759,-0.084532,...,-0.027079,0.108531,-0.076217,-0.012878,-0.154045,0.055077,-0.005913,-0.047781,0.008047,0.002012
2,4,-0.070908,-0.068187,-0.048338,-0.148972,0.046354,0.132966,0.026155,-0.030616,-0.017034,...,0.066733,0.121087,-0.008704,0.005541,-0.010806,0.072195,0.054540,-0.076624,-0.072656,0.062275
3,5,0.015193,0.108114,-0.059156,-0.062374,0.093193,0.095682,0.023337,-0.033058,-0.080769,...,0.148111,0.098194,0.032282,0.187567,0.015080,0.002625,0.079945,-0.008672,0.012438,-0.078997
4,6,0.120460,0.116227,0.073314,-0.092273,-0.034998,0.050711,-0.036398,0.054734,-0.137434,...,0.046523,0.110233,-0.028940,0.135374,0.017890,0.008272,0.142290,0.130462,0.147706,0.051248


In [133]:
dense_patient_feature_df.to_csv('dense_feature.csv', index=False)